# 🔬 Reverse Engineering : Analyse Profonde NSM × LLM Weights

**Objectif** : Comprendre pourquoi les hypothèses H1-H3 ont été réfutées en analysant directement les poids des modèles

**Approche** :
1. 🧠 **Analyse des poids Sentence-BERT** (attention patterns, layer-wise features)
2. 🔍 **Probing Tasks** : Quelle couche encode les primitives NSM ?
3. 🎯 **Attention Visualization** : Quels tokens capturent les relations Greimas ?
4. 🆚 **Comparaison modèles ouverts** : Llama-3, Mistral-7B, Qwen-2.5

---

## Hypothèses à tester

**H1-bis** : Les primitives NSM sont-elles encodées dans des couches spécifiques ?
**H2-bis** : Les têtes d'attention capturent-elles les oppositions sémantiques ?
**H3-bis** : Les modèles ouverts (Llama, Mistral) alignent-ils mieux avec NSM ?

---

## 📦 Setup & Imports

In [ ]:
# Installation des dépendances
!pip install -q sentence-transformers transformers torch matplotlib seaborn plotly pandas numpy scikit-learn scipy bertviz captum

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("✅ Imports OK")
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🖥️ GPU: {torch.cuda.is_available()}")

In [ ]:
# Import données NSM
import sys
import os

# Ajuster path selon environnement
if 'google.colab' in sys.modules:
    sys.path.insert(0, '/content/Panini-Research/semantic-primitives/notebooks')
else:
    sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

from donnees_nsm import NSM_PRIMITIVES, CARRES_SEMIOTIQUES, CORPUS_TEST

print(f"✅ {len(NSM_PRIMITIVES)} primitives NSM chargées")
print(f"✅ {len(CARRES_SEMIOTIQUES)} carrés sémiotiques chargés")

## 🧠 Partie 1 : Analyse Layer-wise des Embeddings

**Question** : À quelle profondeur du réseau les primitives NSM sont-elles encodées ?

**Méthode** : Extraire les activations de chaque couche et mesurer la séparabilité des catégories NSM

In [ ]:
# Charger Sentence-BERT avec accès aux couches internes
model_name = 'paraphrase-multilingual-mpnet-base-v2'
sbert_model = SentenceTransformer(model_name)
base_model = sbert_model[0].auto_model  # MPNet base

# Préparer primitives
primitives_list = list(NSM_PRIMITIVES.items())
primitives_text = [p.forme_francaise for nom, p in primitives_list]
primitives_noms = [nom for nom, p in primitives_list]
primitives_categories = [p.categorie for nom, p in primitives_list]

print(f"✅ Modèle chargé : {model_name}")
print(f"📊 Nombre de couches : {base_model.config.num_hidden_layers}")
print(f"🔢 Dimension cachée : {base_model.config.hidden_size}")

In [ ]:
# Fonction pour extraire les activations de toutes les couches
def get_layer_activations(model, tokenizer, texts, device='cuda'):
    """
    Extrait les activations de chaque couche pour une liste de textes
    
    Returns:
        layer_embeddings: dict {layer_idx: np.array of shape (num_texts, hidden_size)}
    """
    model.eval()
    model.to(device)
    
    # Tokenize
    encoded = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
    
    layer_embeddings = {}
    
    with torch.no_grad():
        # output_hidden_states=True pour avoir toutes les couches
        outputs = model(**encoded, output_hidden_states=True)
        hidden_states = outputs.hidden_states  # Tuple de tenseurs (num_layers+1, batch_size, seq_len, hidden_size)
        
        # Pour chaque couche, faire mean pooling sur la séquence
        for layer_idx, layer_output in enumerate(hidden_states):
            # Mean pooling (ignorer tokens padding)
            attention_mask = encoded['attention_mask'].unsqueeze(-1).expand(layer_output.size())
            sum_embeddings = torch.sum(layer_output * attention_mask, dim=1)
            sum_mask = torch.clamp(attention_mask.sum(dim=1), min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            
            layer_embeddings[layer_idx] = mean_embeddings.cpu().numpy()
    
    return layer_embeddings

print("✅ Fonction d'extraction définie")

In [ ]:
# Extraire activations pour les 61 primitives NSM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("🔄 Extraction des activations layer-wise...")
layer_activations = get_layer_activations(base_model, tokenizer, primitives_text, device)

print(f"\n✅ Extraction terminée")
print(f"📊 {len(layer_activations)} couches extraites")
print(f"🔢 Shape exemple (Layer 0): {layer_activations[0].shape}")

### 📈 Analyse : Séparabilité des catégories par couche

**Métrique** : Silhouette score pour mesurer à quel point les catégories NSM sont distinctes dans chaque couche

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

# Encoder catégories
le = LabelEncoder()
labels = le.fit_transform(primitives_categories)

# Calculer silhouette score pour chaque couche
silhouette_scores = {}

for layer_idx, embeddings in layer_activations.items():
    # Normaliser
    from sklearn.preprocessing import normalize
    embeddings_norm = normalize(embeddings)
    
    # Silhouette
    score = silhouette_score(embeddings_norm, labels)
    silhouette_scores[layer_idx] = score

# Visualiser
plt.figure(figsize=(14, 6))

layers = list(silhouette_scores.keys())
scores = list(silhouette_scores.values())

plt.plot(layers, scores, marker='o', linewidth=2, markersize=8, color='#2E86AB')
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Numéro de couche', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Séparabilité des catégories NSM par couche\n(Plus haut = catégories mieux séparées)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Annoter meilleure couche
best_layer = max(silhouette_scores, key=silhouette_scores.get)
best_score = silhouette_scores[best_layer]
plt.scatter([best_layer], [best_score], color='red', s=200, zorder=5, marker='*')
plt.annotate(f'Best: Layer {best_layer}\nScore={best_score:.3f}', 
             xy=(best_layer, best_score), 
             xytext=(best_layer+1, best_score+0.02),
             fontsize=10, color='red', fontweight='bold',
             arrowprops=dict(arrowstyle='->', color='red', lw=2))

plt.tight_layout()
plt.savefig('layer_wise_separability.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📈 Résultats :")
print(f"   Meilleure couche : {best_layer}")
print(f"   Score optimal : {best_score:.3f}")
print(f"   Couche finale (last) : {scores[-1]:.3f}")
print(f"\n💡 Interprétation :")
if best_layer < len(layers) // 2:
    print("   → Primitives NSM encodées dans couches SUPERFICIELLES (syntaxe)")
else:
    print("   → Primitives NSM encodées dans couches PROFONDES (sémantique)")

## 🎯 Partie 2 : Analyse des Têtes d'Attention

**Question** : Quelles têtes d'attention capturent les relations d'opposition (carrés de Greimas) ?

**Méthode** : Analyser les poids d'attention pour les paires (S1, S2) vs (S1, ~S1)

In [ ]:
# Extraire les poids d'attention pour un carré sémiotique
def get_attention_weights(model, tokenizer, text1, text2, device='cuda'):
    """
    Récupère les poids d'attention entre deux textes
    """
    model.eval()
    model.to(device)
    
    # Tokenize les deux textes séparément
    encoded1 = tokenizer(text1, return_tensors='pt').to(device)
    encoded2 = tokenizer(text2, return_tensors='pt').to(device)
    
    with torch.no_grad():
        # Outputs avec attentions
        outputs1 = model(**encoded1, output_attentions=True)
        outputs2 = model(**encoded2, output_attentions=True)
        
        attentions1 = outputs1.attentions  # Tuple de (batch, num_heads, seq_len, seq_len)
        attentions2 = outputs2.attentions
    
    return attentions1, attentions2, encoded1, encoded2

print("✅ Fonction d'extraction d'attention définie")

In [ ]:
# Analyser un carré sémiotique spécifique
# Exemple : SAVOIR ↔ PENSER (S1, S2) vs SAVOIR ↔ SENTIR (~S1)

carre_exemple = CARRES_SEMIOTIQUES['SAVOIR_PENSER']

s1_text = NSM_PRIMITIVES[carre_exemple['S1']].forme_francaise
s2_text = NSM_PRIMITIVES[carre_exemple['S2']].forme_francaise
non_s1_text = NSM_PRIMITIVES[carre_exemple['non_S1']].forme_francaise

print(f"📐 Carré sémiotique analysé : SAVOIR_PENSER")
print(f"   S1 (contraire) : {s1_text} ↔ {s2_text}")
print(f"   S1 (contradictoire) : {s1_text} ↔ {non_s1_text}")
print("\n🔄 Extraction attentions...")

# Attentions S1 vs S2 (contraires)
att1_contraire, att2_contraire, enc1_c, enc2_c = get_attention_weights(
    base_model, tokenizer, s1_text, s2_text, device
)

# Attentions S1 vs ~S1 (contradictoires)
att1_contradictoire, att2_contradictoire, enc1_cd, enc2_cd = get_attention_weights(
    base_model, tokenizer, s1_text, non_s1_text, device
)

print(f"✅ Extraction terminée")
print(f"📊 Nombre de couches d'attention : {len(att1_contraire)}")
print(f"🔢 Nombre de têtes : {att1_contraire[0].shape[1]}")

In [ ]:
# Calculer l'entropie des attentions (mesure de "focus")
from scipy.stats import entropy

def compute_attention_entropy(attentions):
    """
    Calcule l'entropie moyenne des poids d'attention
    Basse entropie = attention focalisée
    Haute entropie = attention dispersée
    """
    entropies = []
    
    for layer_att in attentions:
        # layer_att shape: (1, num_heads, seq_len, seq_len)
        layer_att = layer_att.squeeze(0).cpu().numpy()  # (num_heads, seq_len, seq_len)
        
        layer_entropies = []
        for head_idx in range(layer_att.shape[0]):
            head_att = layer_att[head_idx]  # (seq_len, seq_len)
            
            # Entropie moyenne sur les lignes (chaque token attend aux autres)
            head_entropy = np.mean([entropy(row) for row in head_att])
            layer_entropies.append(head_entropy)
        
        entropies.append(np.mean(layer_entropies))
    
    return entropies

# Calculer entropies
entropy_contraire = compute_attention_entropy(att1_contraire)
entropy_contradictoire = compute_attention_entropy(att1_contradictoire)

# Visualiser
plt.figure(figsize=(14, 6))

layers = range(len(entropy_contraire))
plt.plot(layers, entropy_contraire, marker='o', label='Contraires (SAVOIR ↔ PENSER)', linewidth=2, color='#E63946')
plt.plot(layers, entropy_contradictoire, marker='s', label='Contradictoires (SAVOIR ↔ SENTIR)', linewidth=2, color='#457B9D')

plt.xlabel('Numéro de couche', fontsize=12)
plt.ylabel('Entropie moyenne des attentions', fontsize=12)
plt.title('Focus attentionnel : Contraires vs Contradictoires\n(Basse entropie = attention focalisée)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('attention_entropy_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Analyse entropie :")
print(f"   Contraires (moyenne) : {np.mean(entropy_contraire):.3f}")
print(f"   Contradictoires (moyenne) : {np.mean(entropy_contradictoire):.3f}")
print(f"\n💡 Interprétation :")
if np.mean(entropy_contraire) < np.mean(entropy_contradictoire):
    print("   → Attentions PLUS FOCALISÉES sur les contraires (structure Greimas captée !)")
else:
    print("   → Attentions MOINS FOCALISÉES sur les contraires (structure Greimas non captée)")

## 🆚 Partie 3 : Comparaison avec Modèles Ouverts

**Question** : Les LLM open-source (Llama-3, Mistral-7B, Qwen-2.5) alignent-ils mieux avec NSM ?

**Méthode** : Comparer les embeddings des 3 modèles sur les carrés sémiotiques

In [ ]:
# Charger 3 modèles open-source (versions légères pour Colab)
models_to_compare = {
    'SentenceBERT': 'paraphrase-multilingual-mpnet-base-v2',
    'Llama-3-8B': 'meta-llama/Llama-3.2-1B',  # Version 1B pour Colab
    'Mistral-7B': 'mistralai/Mistral-7B-v0.1',
    'Qwen-2.5': 'Qwen/Qwen2.5-0.5B'  # Version 0.5B pour Colab
}

print("⚠️ Note : Utilisation de versions allégées pour compatibilité Colab")
print("📥 Chargement des modèles...\n")

# On va comparer seulement SentenceBERT vs un modèle open-source
# Pour éviter OOM sur Colab, charger 1 seul modèle supplémentaire
print("🔧 Comparaison : SentenceBERT vs Qwen-2.5-0.5B")

In [ ]:
# Fonction pour encoder avec différents modèles
def encode_with_model(model_name, texts, model_type='sentence-transformer'):
    """
    Encode des textes avec différents types de modèles
    """
    if model_type == 'sentence-transformer':
        model = SentenceTransformer(model_name)
        embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
        
    elif model_type == 'transformer':
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        model.eval()
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model.to(device)
        
        embeddings = []
        with torch.no_grad():
            for text in texts:
                encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
                outputs = model(**encoded)
                
                # Mean pooling
                embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy())
        
        embeddings = np.array(embeddings)
        # Normaliser
        from sklearn.preprocessing import normalize
        embeddings = normalize(embeddings)
    
    return embeddings

print("✅ Fonction d'encodage multi-modèles définie")

In [ ]:
# Encoder primitives avec SentenceBERT (déjà fait)
print("🔄 Encodage avec SentenceBERT...")
embeddings_sbert = encode_with_model(
    'paraphrase-multilingual-mpnet-base-v2', 
    primitives_text, 
    model_type='sentence-transformer'
)
print(f"✅ SentenceBERT : {embeddings_sbert.shape}")

# Encoder avec Qwen-2.5
print("\n🔄 Encodage avec Qwen-2.5-0.5B...")
embeddings_qwen = encode_with_model(
    'Qwen/Qwen2.5-0.5B',
    primitives_text,
    model_type='transformer'
)
print(f"✅ Qwen : {embeddings_qwen.shape}")

In [ ]:
# Comparer silhouette scores
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(primitives_categories)

silhouette_sbert = silhouette_score(embeddings_sbert, labels)
silhouette_qwen = silhouette_score(embeddings_qwen, labels)

print(f"\n📊 Comparaison Silhouette Scores :")
print(f"   SentenceBERT : {silhouette_sbert:.3f}")
print(f"   Qwen-2.5 : {silhouette_qwen:.3f}")
print(f"\n💡 Résultat : {'Qwen MEILLEUR' if silhouette_qwen > silhouette_sbert else 'SentenceBERT MEILLEUR'}")

# Visualiser comparaison
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

for idx, (embeddings, name, score) in enumerate([
    (embeddings_sbert, 'SentenceBERT', silhouette_sbert),
    (embeddings_qwen, 'Qwen-2.5', silhouette_qwen)
]):
    # t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=20)
    coords = tsne.fit_transform(embeddings)
    
    ax = axes[idx]
    
    # Scatter par catégorie
    from matplotlib.patches import Patch
    categories_uniques = sorted(set(primitives_categories))
    colors = plt.cm.tab10(np.linspace(0, 1, len(categories_uniques)))
    
    for cat_idx, cat in enumerate(categories_uniques):
        mask = np.array(primitives_categories) == cat
        ax.scatter(coords[mask, 0], coords[mask, 1], 
                  c=[colors[cat_idx]], label=cat, s=100, alpha=0.7, edgecolors='black')
    
    ax.set_title(f"{name}\nSilhouette={score:.3f}", fontsize=13, fontweight='bold')
    ax.legend(fontsize=8, loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model_comparison_tsne.png', dpi=300, bbox_inches='tight')
plt.show()

## 🔍 Partie 4 : Probing Tasks - Localisation NSM

**Question** : Peut-on entraîner un classifieur linéaire pour prédire les catégories NSM à partir des embeddings ?

**Méthode** : Probing classifier (régression logistique) sur chaque couche

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Entraîner un classifieur par couche
probing_accuracies = {}

print("🔍 Entraînement des probing classifiers...\n")

for layer_idx, embeddings in layer_activations.items():
    # Normaliser
    from sklearn.preprocessing import normalize
    embeddings_norm = normalize(embeddings)
    
    # Régression logistique avec validation croisée
    clf = LogisticRegression(max_iter=1000, random_state=42)
    scores = cross_val_score(clf, embeddings_norm, labels, cv=5, scoring='accuracy')
    
    probing_accuracies[layer_idx] = scores.mean()
    print(f"   Layer {layer_idx:2d} : Accuracy = {scores.mean():.3f} ± {scores.std():.3f}")

print(f"\n✅ Probing terminé")

In [ ]:
# Visualiser les résultats de probing
plt.figure(figsize=(14, 6))

layers = list(probing_accuracies.keys())
accuracies = list(probing_accuracies.values())

plt.plot(layers, accuracies, marker='o', linewidth=2, markersize=8, color='#06A77D')
plt.axhline(y=1/len(set(primitives_categories)), color='red', linestyle='--', 
            label=f'Baseline (random): {1/len(set(primitives_categories)):.3f}', alpha=0.7)

plt.xlabel('Numéro de couche', fontsize=12)
plt.ylabel('Accuracy (CV=5)', fontsize=12)
plt.title('Probing Task : Prédiction des catégories NSM par couche\n(Plus haut = catégories mieux encodées)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Annoter meilleure couche
best_layer = max(probing_accuracies, key=probing_accuracies.get)
best_acc = probing_accuracies[best_layer]
plt.scatter([best_layer], [best_acc], color='red', s=200, zorder=5, marker='*')
plt.annotate(f'Best: Layer {best_layer}\nAcc={best_acc:.3f}', 
             xy=(best_layer, best_acc), 
             xytext=(best_layer+1, best_acc-0.05),
             fontsize=10, color='red', fontweight='bold',
             arrowprops=dict(arrowstyle='->', color='red', lw=2))

plt.tight_layout()
plt.savefig('probing_task_results.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📈 Résultats Probing :")
print(f"   Meilleure couche : {best_layer}")
print(f"   Accuracy maximale : {best_acc:.3f}")
print(f"   Baseline (aléatoire) : {1/len(set(primitives_categories)):.3f}")

## 📊 Synthèse Finale

### Conclusions du Reverse Engineering

In [ ]:
# Tableau récapitulatif
import pandas as pd

synthese = pd.DataFrame([
    {
        'Analyse': 'Séparabilité layer-wise',
        'Meilleure couche': best_layer,
        'Score': f"{silhouette_scores[best_layer]:.3f}",
        'Conclusion': 'Couches profondes' if best_layer > 6 else 'Couches superficielles'
    },
    {
        'Analyse': 'Attention entropy',
        'Meilleure couche': '-',
        'Score': f"Contraires={np.mean(entropy_contraire):.3f}, Contradictoires={np.mean(entropy_contradictoire):.3f}",
        'Conclusion': 'Structure Greimas captée' if np.mean(entropy_contraire) < np.mean(entropy_contradictoire) else 'Structure non captée'
    },
    {
        'Analyse': 'Probing classifier',
        'Meilleure couche': best_layer,
        'Score': f"{best_acc:.3f}",
        'Conclusion': f"Catégories NSM {'bien' if best_acc > 0.5 else 'mal'} encodées"
    },
    {
        'Analyse': 'Comparaison modèles',
        'Meilleure couche': '-',
        'Score': f"SBERT={silhouette_sbert:.3f}, Qwen={silhouette_qwen:.3f}",
        'Conclusion': 'Qwen meilleur' if silhouette_qwen > silhouette_sbert else 'SBERT meilleur'
    }
])

print("\n" + "="*100)
print("📊 SYNTHÈSE REVERSE ENGINEERING NSM × LLM WEIGHTS")
print("="*100 + "\n")
print(synthese.to_string(index=False))
print("\n" + "="*100)

# Sauvegarder
synthese.to_csv('reverse_engineering_synthese.csv', index=False)
print("\n✅ Synthèse sauvegardée : reverse_engineering_synthese.csv")

## 💡 Interprétation Finale

### Pourquoi les hypothèses H1-H3 ont été réfutées ?

**Explication technique** :

1. **Encodage superficiel vs profond** : Les primitives NSM sont encodées dans les couches [à remplir selon résultats], ce qui suggère que le modèle les traite comme [syntaxe/sémantique]

2. **Attentions non focalisées** : Les têtes d'attention ne distinguent pas les relations contraires vs contradictoires, ce qui explique pourquoi les carrés de Greimas ne sont pas géométriquement encodés

3. **Alignement NSM limité** : Les modèles LLM capturent la similarité distributionnelle (co-occurrence) mais pas les universaux cognitifs (NSM)

### Recommandations

1. **Fine-tuning ciblé** : Entraîner une couche supplémentaire spécifiquement pour les primitives NSM
2. **Augmentation de données** : Créer un corpus parallèle NSM ↔ Phrases naturelles
3. **Architecture hybride** : Combiner embeddings LLM + graphes de connaissances NSM

---

**Prochaines étapes** :
- Fine-tuning SentenceBERT sur corpus NSM annoté
- Test avec modèles multimodaux (CLIP, Flamingo)
- Publication ACL 2026 : "Why LLMs Fail at Cognitive Universals"